In [ ]:
# Null Hypothesis:
# There is no significant difference between the given conditions of measurement OR the probability distributions for all the conditions are the same. (Medians are equal)
# Alternate Hypothesis:
# At least 2 of them differ from each other

In [ ]:
# Test statistics 
# n = total number of subjects/participants.
# k = total number of blocks to be measured.
# Ri = sum of ranks of all subjects for a block i

In [11]:
### Friedman Test with dataframe ###

import os
import sys
from neo4j import GraphDatabase
from dotenv import load_dotenv
import pandas as pd
from scipy.stats import friedmanchisquare

# Load environment variables from .env file
load_dotenv()

def fetch_ratings(uri, user, password, rating_properties):
    try:
        driver = GraphDatabase.driver(uri, auth=(user, password))
        with driver.session() as session:
            # Build the Cypher query to fetch 'name' and all rating properties
            query = f"""
                MATCH (d:Drug)
                RETURN d.name AS name, {', '.join([f'd.{prop} AS {prop}' for prop in rating_properties])}
            """
            result = session.run(query)
            # Collect all data into a list of dictionaries
            data = []
            for record in result:
                row = {key: record.get(key) for key in ['name'] + rating_properties}
                data.append(row)
            return data
    except Exception as e:
        print(f"An error occurred: {e}")
        return []
    finally:
        if 'driver' in locals():
            driver.close()

def clean_and_convert_to_numeric(data):
    # Convert data list of dicts to DataFrame
    df = pd.DataFrame(data)
    
    # Initialize counters
    non_numeric_count = 0
    initial_row_count = len(df)
    

    # Convert rating columns to numeric
    rating_cols = [col for col in df.columns if col != 'name']
    for col in rating_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        non_numeric_count += df[col].isna().sum()
    
    # Identify drugs that will be dropped (those with NaN in rating columns)
    dropped_drugs = df[df[rating_cols].isna().any(axis=1)]['name'].tolist()

    # In the clean_and_convert_to_numeric function, before conversion
    # dropped_drugs_raw = df[df[rating_cols].isna().any(axis=1)]
    # print("Raw data for dropped drugs:")
    # print(dropped_drugs_raw)

    
    # Drop rows with any NaN in rating columns
    df_cleaned = df.dropna(subset=rating_cols)
    
    # Calculate the number of drugs dropped
    dropped_drugs_count = initial_row_count - len(df_cleaned)
    
    print(f"Number of non-numeric or missing values dropped: {non_numeric_count}")
    print(f"Number of drugs dropped due to incomplete data: {dropped_drugs_count}")
    print(f"Drugs dropped: {dropped_drugs}")
    
    # Save the cleaned data to CSV
    df_cleaned.to_csv('ratings_of_all_iterations_used_for_friedman_test.csv', index=False)
    print("Cleaned data saved to 'data_used_for_friedman_test.csv'")
    
    return df_cleaned

def perform_friedman_test(data):
    df_cleaned = clean_and_convert_to_numeric(data)
    
    # Ensure that all columns have the same number of valid data points after cleaning
    rating_cols = [col for col in df_cleaned.columns if col != 'name']
    if len(df_cleaned) > 0 and all(len(df_cleaned[col]) == len(df_cleaned[rating_cols[0]]) for col in rating_cols):
        # Perform Friedman test
        stat, p = friedmanchisquare(*[df_cleaned[col] for col in rating_cols])
    
        print(f"Friedman test statistic: {stat}")
        print(f"p-value: {p}")
    
        if p < 0.05:
            print("The test is significant, indicating that there is a difference in ratings across iterations.")
        else:
            print("The test is not significant, indicating no strong evidence of differences in ratings across iterations.")
    else:
        print("Not enough valid data after cleaning to perform the Friedman test.")

# Connection details
uri = os.getenv("uri")
user = os.getenv("username")
password = os.getenv("password")

# List of rating properties to compare
rating_properties = [
    'rating_0', 'rating_1', 'rating_2', 'rating_3', 'rating_4',
    'rating_5', 'rating_6', 'rating_7', 'rating_8', 'rating_9'
]

# Fetch ratings and perform the Friedman test
data = fetch_ratings(uri, user, password, rating_properties)
if data:  # Check if there's any rating data
    perform_friedman_test(data)


Number of non-numeric or missing values dropped: 4
Number of drugs dropped due to incomplete data: 4
Drugs dropped: ['Lorvotuzumab mertansine', 'MK-5108', 'GLPG-0974', 'alpha-D-glucose-1-phosphate']
Cleaned data saved to 'data_used_for_friedman_test.csv'
Friedman test statistic: 11.59262340206422
p-value: 0.2372612441427373
The test is not significant, indicating no strong evidence of differences in ratings across iterations.
